In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sample=11
count=0
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('D:/python/Open Cv2/haarcascade_frontalface_default.xml')
while count < sample:
    ret,frame=cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    face=face_cascade.detectMultiScale(gray)
    for x,y,w,h in face:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,34,90),2)
        count+=1
        cv2.putText(frame,'Sample no.'+str(count),(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(34,98,23))
        new_img=gray[y:y+h,x:x+w]
        new_img=cv2.resize(new_img,(112,92))
        
        for i in range(1,3):
            cv2.imwrite('D:/python/Open Cv2/dataset/u%d/%d.jpg'%(i,count),new_img)
            
    cv2.imshow('frame',frame)
    key=cv2.waitKey(1)
    if key==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

    
train_data=np.zeros((20,280))
train_label=np.zeros((20))
count=-1
from skimage import feature
import matplotlib.image as mpimg
for i in range(1,3):
    for j in range(1,11):
        count+=1
        path='D:/python/Open Cv2/dataset/u%d/%d.jpg'%(i,j)
        img=mpimg.imread(path)
        fig,hog_img=feature.hog(img,orientations=8,pixels_per_cell=(16,16),
                                     visualize=True,block_norm='L2-Hys',
                                     cells_per_block=(1,1))
        train_data[count,:]=fig.reshape(1,-1)
        train_label[count]=i
from sklearn.svm import SVC
model=SVC(kernel='poly',gamma='scale')
model.fit(train_data,train_label)
import pickle
file=open('D:/python/Open Cv2/SVM.pkl','wb')
pickle.dump(model,file)
        
model=pickle.load(open('D:/python/Open Cv2/SVM.pkl','rb'))
face_cascade=cv2.CascadeClassifier('D:/python/Open Cv2/haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)
while 1:
    ret,frame=cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    face=face_cascade.detectMultiScale(gray)
    for x,y,w,h in face:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(98,255,63),2)
        im_f=gray[y:y+h,x:x+w]
        im_f=cv2.resize(im_f,(112,92))
        feat,hog=feature.hog(im_f,orientations=8,pixels_per_cell=(16,16),
                                     visualize=True,block_norm='L2-Hys',
                                     cells_per_block=(1,1))
        pred=model.predict(feat.reshape(1,-1))
        if pred==1:
          cv2.putText(frame,'Ravi',(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
        if pred==2:
          cv2.putText(frame,'Ravi',(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
    cv2.imshow('detect',frame)
    key=cv2.waitKey(1)
    if key==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()